## STAC API

<img src="https://raw.githubusercontent.com/radiantearth/stac-site/master/images/logo/stac-030-long.png" width="30%"></img>

https://stacspec.org/


### PyStac Client

* https://pystac.readthedocs.io/en/latest/   ( Funktioniert leider nicht mit swisstopo API )

* eine **einfache** Implementation habe ich mit swisstopostac.py gemacht


### Swisstopo STAC

https://www.geo.admin.ch/de/geo-dienstleistungen/geodienste/downloadienste/stac-api.html

Root: https://data.geo.admin.ch/api/stac/v0.9/


**API - Abfrageendpunkte**

| Endpoint | Description |
|----------|---------------|
| /        | Liefert die API-Capabilities |
| /conformance | Info über die Standards, mit denen die API konform ist |
| /collections | Verfügbare Datensätze auflisten (Collections) |
| /collections/{collectionId} | Liefert Metadaten der einzelnen Collection (JSON) |
| /collections/{collectionId}/items | Liefert die Items in der Collection (GeoJSON) |
| /collections/{collectionId}/items/{featureId} |iefert ein einzelnes Item (GeoJSON) |
| /search | Liefert eine Liste von Items, die den Abfrageparametern entsprechen. Ähnlich wie /collections/{collectionId}/items, führt jedoch die Filterung über alle Collections durch |




In [ ]:
import geopandas as gpd
import shapely
import sys

In [ ]:
sys.path.append('..')
import geopandas_stac as stac   # wir sind im Notebook-directory, die Bibliothek ist eine Ebene höher

## Katalog anzeigen

In [ ]:
df_collections = stac.getCollections(cache=False)

df_collections

``id`` zeilenweise ausgeben

In [ ]:
def print_info(row):
    uid = row['id']
    print(uid)
    
r = df_collections.apply(print_info, axis=1)

In [ ]:
df = stac.getFeatures("ch.swisstopo.pixelkarte-farbe-pk50.noscale", cache=True)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
df_assets = stac.genAssets(df)
len(df_assets)

In [ ]:
df_assets.head()

**Varianten** siehe auch: https://www.swisstopo.admin.ch/content/swisstopo-internet/en/swisstopo/documents.download/swisstopo-internet/en/documents/karto-documents/shop/SMRProduktdokumentation_D.pdf

* KREL Farbkombination mit Relief (RGB)
* KOMB Farbkombination ohne Relief (indizierte Farben)
* KGRS Graustufenkombination ohne Relief (indizierte Farben)
* EE Einzelebenen (Bitmap, RELI und GTON als Graustufen)
        

In [ ]:
list(df_assets['variant'].unique())

**Auflösung**

Nur die Variante mit 2.5m pro Pixel

In [ ]:
list(df_assets['gsd'].unique())

**CRS**

Nur EPSG Code 2056 (LV95)

In [ ]:
list(df_assets['proj'].unique())

**Fileformat** / **Filetype**

In [ ]:
list(df_assets['type'].unique())

**Schlussauswahl**

Wir können also z.B. Dataframes für jede Variante erstellen:

In [ ]:
df_kgrs = df_assets.query('variant == "kgrs"') # Farbkombination mit Relief (RGB)
df_krel = df_assets.query('variant == "krel"') # Farbkombination ohne Relief (indizierte Farben)
df_komb = df_assets.query('variant == "komb"') # Graustufenkombination ohne Relief (indizierte Farben)

In [ ]:
df_krel.head(3)

nun müsste man noch die URL auslesen, dies ist in der Hilfsfunktion getUrlList(df)

### Nochmals der gesamte Weg (vereinfacht)


* ``stac.getCollectionList(cache=True)``
* ``df = stac.getAssets("ASSETNAME")``
* Eindeutigkeit Abfragen (z.B. Auflösung, Variante)
* ``urls = stac.getUrlList(df_krel)``


In [ ]:
stac.getCollectionList(cache=True)

In [ ]:
df = stac.getAssets("ch.swisstopo.pixelkarte-farbe-pk50.noscale")

... hier müsste man noch wissen, was eindeutig ist ...

In [ ]:
df_kgrs = df.query('variant == "kgrs"') # Farbkombination mit Relief (RGB)
df_krel = df.query('variant == "krel"') # Farbkombination ohne Relief (indizierte Farben)
df_komb = df.query('variant == "komb"') # Graustufenkombination ohne Relief (indizierte Farben)

In [ ]:
urls = stac.getUrlList(df_krel)

Wir könnten jetzt alles herunterladen...

In [ ]:
urls[0:5]  # Die ersten 5 in der liste

### Räumliche Abfragen

Im Webinterface der Swisstopo haben wir ja folgende Auswahlmöglichkeiten gehabt:

* Rechteck
* Polygon
* Klicken
* Kanton
* Gemeinde
* Ganzer Datensatz

Im Prinzip haben wir jetzt den **ganzen Datensatz**, also wir können recht einfach ein CSV selbst erstellen.


#### Punkt

In [ ]:
# Bundeshaus:
lat = 46.94653998135123
lng = 7.444120726365559

In [ ]:
point = shapely.geometry.Point(lng, lat)
bundeshaus_punkt = gpd.GeoDataFrame(geometry=gpd.GeoSeries(point, crs="epsg:4326"))

In [ ]:
bundeshaus_punkt

In [ ]:
# siehe: https://geopandas.org/gallery/spatial_joins.html

df_krel_bundeshaus = gpd.sjoin(df_krel, bundeshaus_punkt, op='contains')

In [ ]:
urls = stac.getUrlList(df_krel_bundeshaus)
urls

#### Polygon

Polygon: Bern, Olten, Luzern, Interlaken

In [ ]:
s = "POLYGON((7.45147705078125 46.95401192579361,7.84698486328125 46.677710064644344,8.35235595703125 47.00647991252098,7.915649414062499 47.336961408985005,7.45147705078125 46.95401192579361))"

In [ ]:
import shapely.wkt

polygon = shapely.wkt.loads(s)
poly_gpd = gpd.GeoDataFrame(geometry=gpd.GeoSeries(polygon, crs="epsg:4326"))

In [ ]:
# siehe: https://geopandas.org/gallery/spatial_joins.html

df_polygon = gpd.sjoin(df_krel, poly_gpd, op='intersects')

In [ ]:
urls = stac.getUrlList(df_polygon)
urls

#### Kanton, Gemeinde

ist analog Polygon

Für solche Anwendungen reichen generalisierte Polygone. 

Ein solches Shapefile kann z.B. bei https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/geostat/geodaten-bundesstatistik/administrative-grenzen/generalisierte-gemeindegrenzen.html bezogen werden


In [ ]:
kantone = gpd.read_file("daten/gemeindegrenzen/ggg_2021-LV95/shp/g1k21.shp", encoding="utf-8")
kantone = kantone[['KTNR','KTNAME','AREA_HA', 'geometry']]
kantone.head()

In [ ]:
kantone = kantone.to_crs("EPSG:4326")
kantone.head(2)

In [ ]:
poly_gpd = kantone.query('KTNR == 2')
poly_gpd

In [ ]:
df_daten_kantone = gpd.sjoin(df_krel, poly_gpd, op='intersects')

In [ ]:
urls = stac.getUrlList(df_daten_kantone)
urls